In [ ]:
import gaps_online as go
import gaps_online.db as db
import re
from glob import glob
import numpy as np
import math
from collections import defaultdict
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path 
import os

In [ ]:
#load calibrations
calib = go.tof.calibrations.load_calibrations(Path('/data1/nextcloud/cra_data/data/2023_nevis/tof/calibration/2024_03_12/'))

if len(calib.keys()) != 40: print('Oh no! Some calibrations are missing :(')
else: print('Yay! All calibration files were loaded :)')

In [ ]:
#get size of a file or set of files

def get_file_size_in_gb(file_path):
    size_in_bytes = os.path.getsize(file_path)
    size_in_gb = size_in_bytes / (1024**3)  # Convert bytes to gigabytes
    return size_in_gb

In [ ]:
#load paths and populate empty data structures for later use

max_pulse_dict = defaultdict(list)
max_pulses = []

tof_events = []
fixed_path = "/data1/nextcloud/cra_data/data/2023_nevis/tof/142/Run142"


files = glob(f'{fixed_path}_*.tof.gaps')
#files = files [-5:]
print(len(files)) #checking to make sure files was populated

In [ ]:
#basic method to acquire paddle id
paddle_ids = []
for fname in tqdm(files[-2:]):
    data = go.tof.get_tofpackets(fname, filter=go.cxx_api.PacketType.TofEvent)
    
    for ev in (go.cxx_api.TofEvent.from_tofpacket(k) for k in data):
        for rb in ev.get_rbids():
            rb_ev = ev.get_rbevent(rb)
            for h in rb_ev.hits:
                paddle_id = h.paddle_id
                paddle_ids.append(paddle_id)

In [ ]:
def generate_paddle_ids(files):
    for fname in tqdm(files):
        data = go.tof.get_tofpackets(fname, filter=go.cxx_api.PacketType.TofEvent)
        for ev in (go.cxx_api.TofEvent.from_tofpacket(k) for k in data):
            for rb in ev.get_rbids():
                rb_ev = ev.get_rbevent(rb)
                for h in rb_ev.hits:
                    paddle_id = h.paddle_id
                    yield paddle_id

In [ ]:
#not sorted by location
paddle_id_list = []

for paddle_id in generate_paddle_ids(files):
    paddle_id_list.append(paddle_id)

plt.hist(paddle_ids)
plt.title('Paddle Occupancy')
plt.xlabel('paddle_id')
plt.ylabel('count')
plt.show()

In [ ]:
cube_top = range(1, 13)
cube_bottom = range(13, 25)
cube_side = range(25, 57)
corner = range(57, 61)
cortina = range(109, 161)
umbrella = range(61, 109)

# cube_all = []
# cube_all.extend(cube_top)
# cube_all.extend(cube_bottom)
# cube_all.extend(corner)

In [ ]:
# cube_top_evts = []
# cube_bottom_evts = []
# cube_side_evts = []
# corner_evts = []
cortina_evts = []
# umbrella_evts = []

for paddle_id in generate_paddle_ids(files):
    if paddle_id in cube_top:
        #cube_top_evts.append(paddle_id)
        continue
    elif paddle_id in cube_bottom:
        continue
        #cube_bottom_evts.append(paddle_id)
    elif paddle_id in cube_side:
        continue
        #cube_side_evts.append(paddle_id)
    elif paddle_id in corner:
        continue
        #corner_evts.append(paddle_id)
    elif paddle_id in cortina:
        cortina_evts.append(paddle_id)
    elif paddle_id in umbrella:
        continue
        #umbrella_evts.append(paddle_id)

In [ ]:
lists = [cube_top_evts, cube_bottom_evts, cube_side_evts, corner_evts, cortina_evts, umbrella_evts]
labels = ['Cube Top', 'Cube Bottom', 'Cube Side', 'Corner', 'Cortina', 'Umbrella']

# Create a figure with subplots
fig, axes = plt.subplots(nrows=len(lists), figsize=(8, 6))

# Plot histograms for each list
for ax, data, label in zip(axes, lists, labels):
    ax.hist(data)  # Adjust the number of bins as needed
    ax.set_title(label)
    ax.set_xlabel('paddle_id')
    ax.set_ylabel('count')

# Adjust layout
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
panels = []

all_paddles = db.tof_paddle_manifest()

for paddle_id in generate_paddle_ids(files):
    panel = all_paddles[paddle_id]['panel'].panel_id
    panels.append(panel)

In [ ]:


plt.hist(cube_top_evts)
plt.title('Cube Top')
plt.xlabel('paddle_id')
plt.ylabel('count')
plt.show()

In [ ]:
plt.hist(cube_side_evts)
plt.title('Cube Side')
plt.xlabel('paddle_id')
plt.ylabel('count')
plt.show()

In [ ]:
plt.hist(cube_bottom_evts)
plt.title('Cube Bottom')
plt.xlabel('paddle_id')
plt.ylabel('count')
plt.show()

In [ ]:
plt.hist(corner_evts)
plt.title('Corner')
plt.xlabel('paddle_id')
plt.ylabel('count')
plt.xticks(np.arange(57, 61, 1))
plt.show()

In [ ]:


plt.hist(cortina_evts, bins = 50)
plt.title('Cortina')
plt.xlabel('paddle_id')
plt.ylabel('count')
plt.xlim(110, 160)

# Set ticks for every value
plt.xticks(np.arange(110, 161, 1))
plt.gca().set_xticklabels([int(x) if x % 10 == 0 else '' for x in plt.gca().get_xticks()])


plt.show()


In [ ]:
plt.hist(umbrella_evts)
plt.title('Umbrella')
plt.xlabel('paddle_id')
plt.ylabel('count')
plt.show()